In [1]:
#Importing required packages.


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost
import eli5
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import joblib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [2]:
df_fake=pd.read_csv(r'C:\Users\DELL\Downloads\Fake-News-Project--1-\Fake News Project\Fake.csv')
df_true = pd.read_csv(r'C:\Users\DELL\Downloads\Fake-News-Project--1-\Fake News Project\True.csv')

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake["class"] = 0
df_true['class'] = 1

In [6]:
df_fake.shape ,df_true.shape

((23481, 5), (21417, 5))

In [7]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
    
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [8]:
df_fake.shape , df_true.shape

((23471, 5), (21407, 5))

In [9]:
df_fake_manual_testing['class'] = 0
df_true_manual_testing['class'] = 1

In [10]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


### now merge the both column i.e fake and true column

In [12]:
df_merge= pd.concat([df_fake,df_true], axis = 0)
df_merge.head(5)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [13]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

### let's remove unwanted column

In [14]:
df = df_merge.drop(['title','subject','date'], axis = 1)

In [15]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [16]:
df = df.sample(frac = 1)

In [17]:
df.head()

,text,class
4186,Fox News host Dana Perino is finally so sick a...,0
236,WASHINGTON (Reuters) - U.S. President Donald T...,1
2634,WASHINGTON (Reuters) - President Donald Trump ...,1
15982,BRUSSELS (Reuters) - At 70 euros ($82) a night...,1
3424,WASHINGTON (Reuters) - The top Democrat in the...,1


In [18]:
df.reset_index(inplace=True)
df.drop(['index'],axis=1 , inplace=True)

In [19]:
df.columns

Index(['text', 'class'], dtype='object')

In [20]:
df.head()

,text,class
0,Fox News host Dana Perino is finally so sick a...,0
1,WASHINGTON (Reuters) - U.S. President Donald T...,1
2,WASHINGTON (Reuters) - President Donald Trump ...,1
3,BRUSSELS (Reuters) - At 70 euros ($82) a night...,1
4,WASHINGTON (Reuters) - The top Democrat in the...,1


get the function for processing the text

In [21]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+','',text)
    text = re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text = re.sub('\n','' , text)
    text = re.sub('\w*\d\w*','', text)
    return text

In [22]:
import re

In [23]:
df['text'] = df['text'].apply(wordopt)

## divide the target and features column

In [24]:
X = df['text']
Y = df['class']

## spliting the data into training and testing

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.25)

In [26]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

## Logestic Regresssion

In [27]:
LR = LogisticRegression()
LR.fit(xv_train, Y_train)

LogisticRegression()

In [28]:
pred_lr = LR.predict(xv_test)

In [29]:
LR.score(xv_test, Y_test)

0.9844919786096257

In [30]:
print(classification_report(Y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5909
           1       0.98      0.99      0.98      5311

    accuracy                           0.98     11220
   macro avg       0.98      0.98      0.98     11220
weighted avg       0.98      0.98      0.98     11220



## Decision Tree classifier

In [31]:
DT = DecisionTreeClassifier()
DT.fit(xv_train,Y_train)

DecisionTreeClassifier()

In [32]:
pred_dt = DT.predict(xv_test)

In [33]:
DT.score(xv_test,Y_test)

0.9952762923351158

In [34]:
print(classification_report(Y_test,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5909
           1       1.00      0.99      1.00      5311

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## Gradient boosting classifier

In [35]:
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train,Y_train)

GradientBoostingClassifier(random_state=0)

In [36]:
pred_gb = GB.predict(xv_test)

In [37]:
GB.score(xv_test,Y_test)

0.9946524064171123

In [38]:
print(classification_report(Y_test,pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5909
           1       0.99      1.00      0.99      5311

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Random forest classsifier

In [39]:
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train,Y_train)

RandomForestClassifier(random_state=0)

In [40]:
pred_rf = RF.predict(xv_test)

In [41]:
RF.score(xv_test,Y_test)

0.9901069518716578

In [42]:
print(classification_report(Y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5909
           1       0.99      0.99      0.99      5311

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [48]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n ==1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"]= new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                            output_lable(pred_DT[0]),
                                                                                                            output_lable(pred_GB[0]),
                                                                                                            output_lable(pred_RF[0])))

## put any news from internet to find it is fake or real...which model will allow to find this.

In [53]:
news = str(input())
manual_testing(news)

Ukrainian President Volodymyr Zelensky, who has called for a boycott of the 2024 Paris Olympics over the possible participation of Russian athletes, said Friday their presence would be a "manifestation of violence." "The mere presence of representatives of the terrorist state is a manifestation of violence and lawlessness," Zelensky told a virtual summit of sports ministers chaired from London.  "And it cannot be covered up with some pretended neutrality or a white flag," Zelensky said.  PauseUnmute Fullscreen The International Olympic Committee said last month it was exploring a "pathway" to allow Russian and Belarusian competitors to take part in the Paris Olympics, under a neutral flag.  Ukraine has reacted furiously, threatening to pull out of the Games and Zelensky argued that Russian athletes had "no place" in Paris.  "If the Olympic sports were killings and missile strikes, then you know which national team would occupy the first place," he added.  Zelensky argued that not many 

In [50]:
news = str(input())
manual_testing(news)

UN Calls For "Immediate Ceasefire" In Syria To Facilitate Earthquake Aid.


LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News
